In [1]:
import numpy as np
import pandas as pd
import os
from os.path import join
from os.path import exists

import json
import requests
import xmltodict
import re
import time

In [2]:
dataDir = '../data'

# Process the COVID-19 articles retrieved from PubMed

Articles retrieved via `getArticles` notebook

In [3]:
arts = []
with open(join(dataDir, 'articleMetadata.json')) as f:
    for i,line in enumerate(f):
        thisArt = json.loads(line)
        if 'isValid' in thisArt.keys():
            arts.append(thisArt)

## Amend/Clean article metadata

In [4]:
# Complete number of authors and affiliations from the metadata for each article
def get_nAuthors(md):
    if 'nAuthors' in md.keys():
        return md['nAuthors']
    if 'authors' not in md.keys():
        return 0
    if md['authors'] == None:
        return 0
    return len(md['authors']['authors'])

def get_nLocations(md):
    if 'nLocations' in md.keys():
        return md['nLocations']
    if 'authors' not in md.keys():
        return 0
    if md['authors'] == None:
        return 0
    return len(md['authors']['affiliations'])

# Clean the location names
def cleanLocations(md):
    if 'authors' not in md.keys():
        return None
    if md['authors'] == None:
        return None
    for loc in md['authors']['affiliations']:
        loc['address'] = cleanLocationString(loc['address'])
    return md

def cleanLocationString(locStr):
    """ Some location names have spaces b e t w e e n every character. This'll fix it """
    rexp = '(\S\s)+\S(?=\s{1,})'
    matches = [x for x in re.finditer(r'(\S\s)+\S(?=\s{1,})', locStr)]  # see if there are any matches
    if len(matches) == 0:
        return locStr

    # search and replace once for each match
    for i in matches:
        thisMatch = re.search(r'(\S\s)+\S(?=\s{1,})', locStr)   # find the match, along with start and end position from current string
        startIdx = thisMatch.start()
        endIdx = thisMatch.end()

        cleaned = thisMatch.group().replace(' ', '')     # remove the spaces from the match
        locStr = locStr[:startIdx] + cleaned + locStr[endIdx:]  # insert the cleaned word back into the original string

    # replace any 2 or more whitespaces with just 1 whitespace
    locStr = re.sub(r"\s{2,}", " ", locStr)

    return locStr

In [5]:
for art in arts:
    art['nAuthors'] = get_nAuthors(art)
    art['nLocations'] = get_nLocations(art)
    art = cleanLocations(art)

## Summarize Articles

In [6]:
nArts = len(arts)
print('{}: total articles'.format(nArts))
print('--------------')

# count articles with missing fields
summary = {
    'Authors': len([x for x in arts if x['nAuthors'] == 0]),
    'Locations': len([x for x in arts if x['nLocations'] == 0]),
    'Title': len([x for x in arts if x['title'] is None]),
    'Date': len([x for x in arts if x['pubDates'] is None])
}

for k,v in summary.items():
    print('{}: no {}'.format(v,k))

88797: total articles
--------------
4187: no Authors
7083: no Locations
65: no Title
3280: no Date


# Update unique locations

Use the latest set of articles to update the Unique Locations and GeoCodes databases

**Unique Locations** database contains the set of unique address strings found across all affiliations listed in the article metadata. Each address string is associated with a unique location id (`locID`) as well as processed address information and a `geoID` (where applicable) that links the locID to a particular geocode entry in the **cityGeocodes** database

## Tools

#### Preprocess Addresses 
Use the python bindings for the [libpostal library](https://github.com/openvenues/libpostal) to attempt to extract structured address details from the unstructured strings associated with each location

In [7]:
from postal.parser import parse_address

In [8]:
def processAddress(addrStr):
    """parse the given address string. return structured address components"""
    addr = {
        'origAddr': addrStr,
        'city': None,
        'state': None,
        'country': None
    }
    
    # parse the address using libpostal
    addrParts = parse_address(addrStr)
    
    # Find the first instance of city, state, or country in parsed address. 
    # Given that addresses are typically written in descending order of geographic specificity,
    # iterate through parts list backwards in case the parser has returned 2 or more instances of
    # city or county components. In these cases, the latter instance is usually the one we want. 
    for p in reversed(addrParts):
        if p[1] == 'city':
            addr['city'] = p[0]
            break
    
    for p in reversed(addrParts):
        if p[1] == 'state':
            addr['state'] = p[0]
            break
            
    for p in reversed(addrParts):
        if p[1] == 'country':
            addr['country'] = p[0]
            break
            
    return addr

#### Geocode Addresses

In [9]:
from geopy import GoogleV3
from geopy.extra.rate_limiter import RateLimiter

In [10]:
def getAddressCmpt(rawGeo, cmpt):
    """
    return the specified address component from the raw geo
    'cmpt' expected to be one of ['locality', 'country', 'administrative_area_level_1']
    """
    addrParts = rawGeo['address_components']
    for p in addrParts:
        if cmpt in p['types']:
            return p['long_name']
    return None

def getAddrCmptFromID(geoID, cmpt):
    """return the desired address component associated with the desired geoID"""
    # find the geo
    for r in rawGeo:
        if r['place_id'] == geoID:
            thisGeo = r
            break
    
    # return the desired field
    return getAddressCmpt(thisGeo, cmpt)

In [11]:
with open('googleAPI_key.txt', 'r') as f:
    apiKey = f.read().rstrip('\n')
    
# instantiate Google Geocoder
gmap = GoogleV3(api_key=apiKey)

### Summarize current location info from article metadata dataset

this step currently takes ~5min to run

In [12]:
locs = []
for art in arts:
    if art['nLocations'] > 0:
        artLocs = art['authors']['affiliations']
        locs = locs + [x['address'] for x in artLocs]

# get list of unique locations
uniqLocStrs = list(set(locs))

print('{} total locations\n'.format(len(locs)))
print('{} unique location strings across all affiliations'.format(len(uniqLocStrs)))

303288 total locations

254887 unique location strings across all affiliations


### Create list of new addresses not currently in Unique Locations database

In [13]:
# Open (or create) the unique locations database
uniqLocs_fname = join(dataDir, 'uniqueLocations.csv')
if exists(uniqLocs_fname):
    uniqLocs_df = pd.read_csv(uniqLocs_fname)
else:
    uniqLocs_df = pd.DataFrame(columns=['locID', 'origAddr', 'locality', 'admArea_1', 'country', 'geoID'])
    

# Figure out which locations are already in the database
existingLocs = uniqLocs_df['origAddr']
newLocs = list(set(uniqLocStrs).difference(set(existingLocs)))

# remove the empty string from newLocs (uniqLocs reps this as "NaN", thus won't be in the existingLocs list)
newLocs.remove('')

print('{} new locations not found already in unique locations db'.format(len(newLocs)))

24254 new locations not found already in unique locations db


In [14]:
newLocs = list(map(lambda x: {'origAddr': x}, newLocs))

# assign a unique location ID to each new location, numbering up from however many locations already exist
startIdx = uniqLocs_df.shape[0]
for i,loc in enumerate(newLocs, start=1):
    locID = 'loc{}'.format(str(startIdx + i).zfill(9))
    loc['locID'] = locID

### Get GeoCodes for new addresses

Due to the cost of hitting Google's geocode API with hundreds of thousands of location strings, keeping the geocodes at the level of cities-only for now

This step will create or update 2 different databases:

* **cityGeocodes**:   
    a table of geocoded location data. one entry for every unique city
    
* **geoSearchStrs**: a table linking given search strings to a specific geoID in the cityGeocodes table. Due to how searches are constructed, multiple different search strings may link to the same phyical geo location

In [15]:
# Open (or create) the city geocodes database
geo_fname = join(dataDir, 'cityGeocodes.csv')
if exists(geo_fname):
    geo_df = pd.read_csv(geo_fname)
else:
    geo_df = pd.DataFrame(columns=['geoID', 'formattedAddr', 'lat', 'lng', 'locality', 'admArea_1', 'country'])

# Open (or create) the geocodes search string's database
searchStrs_fname = join(dataDir, 'geoSearchStrs.csv')
if exists(searchStrs_fname):
    searchStrs_df = pd.read_csv(searchStrs_fname)
else:
    searchStrs_df = pd.DataFrame(columns=['searchStr', 'geoID'])

# Iterate through new locations. If geoID already exists, use it. If not, get geoID and geocode info
# Build a location object that can be appended to the unique locations database
print('Checking geocodes for {} addresses'.format(len(newLocs)))
missedLocs = []
for i,loc in enumerate(newLocs):
    if i % 50 == 0: print('{}'.format(i), end=', ')  # progress update
        
    # add the remaining missing fields to loc object, initialize with None
    # Note: loc is updated in place
    loc['locality'] = np.nan
    loc['admArea_1'] = np.nan
    loc['country'] = np.nan
    loc['geoID'] = np.nan
    
    ## turn it into an address using libpostal
    addr = processAddress(loc['origAddr'])
    if addr['city'] is None:
        continue              # geocoding at the level of city, so need there to be a "city" in the address
    
    # build a search string based on city, state, country (if each exists)
    searchStr = ' '.join([addr[x] for x in ['city', 'state', 'country'] if addr[x] is not None])
    
    # check if this searchStr already exists in the searchStr database. If so, skip geocoding it again
    if searchStr in searchStrs_df['searchStr'].values:
        geoID = searchStrs_df[searchStrs_df['searchStr'] == searchStr]['geoID'].item()        
        thisGeo = geo_df[geo_df['geoID'] == geoID]
        if thisGeo.empty:
            continue
        for field in ['geoID', 'locality', 'admArea_1', 'country']:
            loc[field] = thisGeo[field].item()

        continue
    
    ###### GEOCODE PHASE 1 #####################################################################
    # use search string to get geocode info from google 
    # Parse the 'locality', 'admin_area_level_1', and 'country' from the results, which
    # will be used to see if this location already exists in the cityGeocodes database
    # (e.g. if this search string refers to a different address in the same city)
    time.sleep(.03)       # avoid getting blocked. max 50/s
    try:
        geo = gmap.geocode(searchStr, exactly_one=True)
    except Exception as e:
        print(str(e))
        missedLocs.append(loc)  # save the missed location
        continue
    
    # if unable to geocode this searchstr, add the search to searchStrs db and skip to next loc
    if geo is None:
        searchStrs_df = searchStrs_df.append({'searchStr': searchStr, 'geoID': None}, ignore_index=True)
        continue
        
    # check if this geo's LOCALITY-ADMINAREA_1-COUNTRY already exists in the cities database
    locality = getAddressCmpt(geo.raw, 'locality')
    admArea_1 = getAddressCmpt(geo.raw, 'administrative_area_level_1')
    country = getAddressCmpt(geo.raw, 'country')
    
    matching_geo = geo_df[(geo_df['locality'] == locality) & (geo_df['admArea_1'] == admArea_1) & (geo_df['country'] == country)]
    if not matching_geo.empty:
        matching_geo = matching_geo.iloc[0]
            
        # get the details for this location from the existing entry in city geocodes
        for field in ['locality', 'admArea_1', 'country', 'geoID']:
            loc[field] = matching_geo[field]
            
        # pair this geoID to this search string in the searchStrs db
        searchStrs_df = searchStrs_df.append({
            'searchStr': searchStr, 
            'geoID': matching_geo['geoID']}, 
        ignore_index=True)
        
        # save searchStr database
        searchStrs_df.to_csv(searchStrs_fname, index=False)
        
        # skip to next location
        continue
    
    ###### GEOCODE PHASE 2 ##################################################################
    # since this LOCALITY-ADMINEAREA_1-COUNTRY not already in cities database,  
    # submit a new GEOCODE request, using the '<locality> <admin area> <county>' as a search string. 
    # Parse the response to get the GEOID, LAT, LNG, FORMATTED ADDRESS, LOCALITY, ADM AREA 1, COUNTRY 
    # of this new location. This will return geocode info specific to this city. Add to the cities database
    
    # build a new search string based on locality, admin area 1, country (if each exists)
    citySearchStr = ' '.join([x for x in [locality, admArea_1, country] if x is not None])
    
    # geocode
    time.sleep(.03)       # avoid getting blocked. max 50/s
    try:
        geo = gmap.geocode(citySearchStr, exactly_one=True)
    except Exception as e:
        print(str(e))
        missedLocs.append(loc)  # save the missed address
        continue
        
    # if unable to geocode this citySearchStr, add the original search to searchStrs db and skip to next loc
    if geo is None:
        searchStrs_df = searchStrs_df.append({'searchStr': searchStr, 'geoID': None}, ignore_index=True)
        continue
    
    # must match to a locality
    locality = getAddressCmpt(geo.raw, 'locality')
    if locality is None:
        searchStrs_df = searchStrs_df.append({'searchStr': searchStr, 'geoID': None}, ignore_index=True)
        continue

    # add this entry to the database
    geo_obj = {
        'geoID': geo.raw['place_id'],
        'lat': geo.latitude,
        'lng': geo.longitude,
        'formattedAddr': geo.raw['formatted_address'],
        'locality': locality,
        'admArea_1': getAddressCmpt(geo.raw, 'administrative_area_level_1'),
        'country': getAddressCmpt(geo.raw, 'country'),
    }
    
    # append this geo obj to the city geocodes db
    geo_obj = {k: np.nan if not v else v for k,v in geo_obj.items()}  # convert Nones to NaN
    geo_df = geo_df.append(geo_obj, ignore_index=True)
    
    # update all fields for this location object
    for field in ['locality', 'admArea_1', 'country', 'geoID']:
        loc[field] = geo_obj[field]
    
    # pair this geoID with the original search string
    searchStrs_df = searchStrs_df.append({
        'searchStr': searchStr, 
        'geoID': geo.raw['place_id']
    }, ignore_index=True)
    
    # save (not efficient, but worth it if checking thousands of locations)
    geo_df = geo_df.drop_duplicates(subset='geoID')
    geo_df.to_csv(geo_fname, index=False)
    searchStrs_df.to_csv(searchStrs_fname, index=False)
    raw_fname = join(dataDir, 'rawGeoResponses.json')
    with open(raw_fname, 'a', encoding='utf-8') as outfile:
        json.dump(geo.raw, outfile, ensure_ascii=False)
        outfile.write('\n')
    
print('\n Missed {} locations'.format(len(missedLocs)))

Checking geocodes for 24254 addresses
0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 2850, 2900, 2950, 3000, 3050, 3100, 3150, 3200, 3250, 3300, 3350, 3400, 3450, 3500, 3550, 3600, 3650, 3700, 3750, 3800, 3850, 3900, 3950, 4000, 4050, 4100, 4150, 4200, 4250, 4300, 4350, 4400, 4450, 4500, 4550, 4600, 4650, 4700, 4750, 4800, 4850, 4900, 4950, 5000, 5050, 5100, 5150, 5200, 5250, 5300, 5350, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000, 6050, 6100, 6150, 6200, 6250, 6300, 6350, 6400, 6450, 6500, 6550, 6600, 6650, 6700, 6750, 6800, 6850, 6900, 6950, 7000, 7050, 7100, 7150, 7200, 7250, 7300, 7350, 7400, 7450, 7500, 7550, 7600, 7650, 7700, 7750, 7800, 7850, 7900, 7950, 8000, 8050, 8100, 8150, 8

### Add the newly processed locations to the unique location strings database

In [16]:
uniqLocs_df = uniqLocs_df.append(newLocs, ignore_index=True)

In [17]:
# save
uniqLocs_df.to_csv(uniqLocs_fname, index=False)

# Add location IDs to affiliations for all articles

In [18]:
def getLocID(addrStr):
    """ Return the location ID and valid geocode flag based on addr string"""
    if addrStr == '':
        loc = uniqLocs_df[uniqLocs_df['origAddr'].isnull()].iloc[0]
        return {'locID': loc['locID'], 'hasGeo': False}
    
    loc = uniqLocs_df[uniqLocs_df['origAddr'] == addrStr]
    if loc.empty:
        return {'locID': None, 'hasGeo': False}
    
    # get locID and check for valid geoID
    locID = loc['locID'].item()
    if pd.isnull(loc['geoID'].item()) or loc['geoID'].item() is None:
        return {'locID': locID, 'hasGeo': False}
    else:
        hasGeo = True
        geoID = loc['geoID'].item()
        return {'locID': locID, 'hasGeo': True, 'geoID': geoID}
    

def getArtLocs(art):
    """process the given article to get all locations for all affiliations"""
    locsWithGeo = 0
    for loc in art['authors']['affiliations']:
        if 'locID' not in loc.keys():
            locInfo = getLocID(loc['address'])
            loc['locID'] = locInfo['locID']
            loc['hasGeo'] = locInfo['hasGeo']
        if loc['hasGeo']:
            locsWithGeo += 1

    art['nLocsWithGeo'] = locsWithGeo
    
    return art
    

This step takes a while to run (~1hr)

In [19]:
# loop over all articles to get location for all affiliations
for art in arts:
    locsWithGeo = 0
    for loc in art['authors']['affiliations']:
        #if 'locID' not in loc.keys():
        locInfo = getLocID(loc['address'])
        loc['locID'] = locInfo['locID']
        loc['hasGeo'] = locInfo['hasGeo']
        if loc['hasGeo']:
            loc['geoID'] = locInfo['geoID']
            locsWithGeo += 1

    art['nLocsWithGeo'] = locsWithGeo

### save processed articles back to disk

In [20]:
with open(join(dataDir, 'articleMetadata.json'), 'w', encoding='utf-8') as f:
    for art in arts:
        json.dump(art, f, ensure_ascii=False)
        f.write('\n')

### Summarize Articles with Locations

In [21]:
totalArts = len(arts)
multiLocs = len([x for x in arts if x['nLocations'] >= 2])
multiLocsWithGeo = len([x for x in arts if x['nLocsWithGeo'] >= 2])

print('{} total articles'.format(totalArts))
print('({}%) {} w/ at least 2 affiliations'.format(np.round(multiLocs/totalArts*100, 2), multiLocs))
print('({}%) {} w/ at least 2 geocoded affiliation'.format(np.round(multiLocsWithGeo/totalArts * 100, 2), multiLocsWithGeo))

88797 total articles
(68.08%) 60453 w/ at least 2 affiliations
(53.35%) 47371 w/ at least 2 geocoded affiliation


In [22]:
# calculate the total number of collaborations across all articles with geocoded affiliations
collabs = 0
for art in arts:
    if art['nLocsWithGeo'] >=2:
        nLocs = art['nLocsWithGeo']
        collabs += nLocs * (nLocs-1) / 2  # calculate unique collabs

print('{} total geocoded collaborations'.format(int(collabs)))

1005369 total geocoded collaborations


## Spot check the unique locations for accuracy

In [132]:
testLocs = uniqLocs_df.sample(100)
testLocs.to_csv(join(dataDir, 'testLocations.csv'), index=False)